In [ ]:
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

import pickle
from tensorflow.python.keras import backend as K
from keras.models import load_model
import pandas as pd
import h5py
import math
from keras.models import Sequential
from keras.layers import Concatenate,AveragePooling3D,AveragePooling2D,ConvLSTM2D,Conv2DTranspose,Conv3DTranspose,MaxPooling3D,Flatten,Dropout,Dense,TimeDistributed,Conv3D,Conv2D,MaxPooling2D,AveragePooling3D,BatchNormalization
import keras
from tensorflow.python.client import device_lib
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from keras import backend as K
Mon_matrix=[-1,'T7-FT9',-1,-1,'FP1-F7','F7-T7','T7-P7','P7-O1','FP1-F3','F3-C3','C3-P3','P3-O1',-1,'FZ-CZ','CZ-PZ',-1,
                'FP2-F4', 'F4-C4','C4-P4','P4-O2','FP2-F8','F8-T8','T8-P8-0','P8-O2', -1,'FT10-T8', -1, -1]
Mon_matrix_col_no=4
sec_frame = 20
freq_seg=0.5 #freq domain 기본 간격

Mon_matrix_row_no=5 #row는 4로 고정
freq_upper_range=40 #freq upper range Hz

freq_n=int(freq_upper_range/freq_seg)#일단 현재는 40
with open('pickle_file/information.pickle', 'rb') as f:
    information_dict = pickle.load(f)
with open('pickle_file/weight_dict.pickle', 'rb') as f:
    weight_dict = pickle.load(f)
with open('pickle_file/weight_mul_dict.pickle', 'rb') as f:
    weight_mul_dict = pickle.load(f)
with open('pickle_file/weight_div_dict.pickle', 'rb') as f:
    weight_div_dict = pickle.load(f)

with open('pickle_file/weight_dict_revise.pickle', 'rb') as f:
    weight_dict_revise = pickle.load(f)
data_list  = []
for i in range(1,24):    
    data_list.append(str(i))
import time
grid_list = []
for i in range(4):
    for j in range(5):
        grid_list.append((i,j))
freq_n = 15
new_channel_location=['FP1-F7','FP1-F3','FP2-F4','FP2-F8','F7-T7','F3-C3','F4-C4','F8-T8','T7-FT9',
                     'FZ-CZ','CZ-PZ','FT10-T8','T7-P7','C3-P3','C4-P4','T8-P8','P7-O1','P3-O1','P4-O2','P8-O2']

In [ ]:
keylist=list(information_dict.keys())
sz_dict = {}
for j in range(0,24):   
    sz_dict[j+1] = {}
    start_list = []
    sz_len_list = []  
    for k in range(len(information_dict[keylist[j]]['new_sz_info_list'])): 
        sz_edf_num=information_dict[keylist[j]]['new_sz_info_list'][k][-1]       
        before_time = 0
        for ti in range(information_dict[keylist[j]]['new_sz_info_list'][k][-1]-1):
            before_time = before_time + information_dict[keylist[j]]['time'][ti]
        start = int(information_dict[keylist[j]]['new_sz_info_list'][k][0] + before_time)   
        #print(start)
        start_list.append(start)
        sz_len_list.append(information_dict[keylist[j]]['new_sz_info_list'][k][1]-information_dict[keylist[j]]['new_sz_info_list'][k][0])
    sz_dict[j+1]['start'] = start_list
    sz_dict[j+1]['len'] = sz_len_list

In [ ]:
sz_dict

In [ ]:
keylist=list(information_dict.keys())
sz_dict = {}
for j in range(0,24):   
    sz_dict[j+1] = {}
    start_list = []
    sz_len_list = []
    #h5f=h5py.File('/MIT_data/source/h5_file/stft/'+'chb'+str(keylist[j])+'_'+'data.h5','r')
    #data=h5f['all_data/data']     
   
    for k in range(len(information_dict[keylist[j]]['new_sz_info_list'])): 
        sz_edf_num=information_dict[keylist[j]]['new_sz_info_list'][k][-1]       
        before_time = 0
        for ti in range(information_dict[keylist[j]]['new_sz_info_list'][k][-1]-1):
            before_time = before_time + information_dict[keylist[j]]['time'][ti]
        start = int(information_dict[keylist[j]]['new_sz_info_list'][k][0] + before_time)   
        #print(start)
        start_list.append(start)
        sz_len_list.append(information_dict[keylist[j]]['new_sz_info_list'][k][1]-information_dict[keylist[j]]['new_sz_info_list'][k][0])
    sz_dict[j+1]['start'] = start_list
    sz_dict[j+1]['len'] = sz_len_list

In [ ]:
def make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k] not in include:  
            #print(data_list[k])
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size)) + 1
    return score
def second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 train data(n-1명)를 생성하는 코드   
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] not in include:  
                
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                        #print(str(index))                
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:] * 10000
                        tmp_weight=weight_tmp[index:index+batch_size+sec_frame]
                        tmp_y=test_y[index:index+batch_size+sec_frame,:]     
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we)                   
                        #print('     '+str(ge_x.shape))  
                        #print('     '+str(ge_y.shape))  
                        #print('     '+str(ge_we.shape))  
                        index  = index + batch_size
                        yield ge_x,ge_y,ge_we    
                                                
                    else:   
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:size,:,:]
                        tmp_weight=weight_tmp[index:size]
                        tmp_y=test_y[index:size,:]
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we) 
                        print('   '+str(len(ge_x))) 
                        yield ge_x,ge_y,ge_we   
                        break

In [ ]:
def val_make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k] in include:  
            #print(data_list[k])
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size)) + 1
    return score
def val_second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 train data(n-1명)를 생성하는 코드   
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] in include:  
                
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                        #print(str(index))                
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:] * 10000
                        tmp_weight=weight_tmp[index:index+batch_size+sec_frame]
                        tmp_y=test_y[index:index+batch_size+sec_frame,:]     
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we)                   
                        #print('     '+str(ge_x.shape))  
                        #print('     '+str(ge_y.shape))  
                        #print('     '+str(ge_we.shape))  
                        index  = index + batch_size
                        yield ge_x,ge_y,ge_we    
                                                
                    else:   
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:size,:,:]
                        tmp_weight=weight_tmp[index:size]
                        tmp_y=test_y[index:size,:]
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we) 
                        print('   '+str(len(ge_x))) 
                        yield ge_x,ge_y,ge_we   
                        break

In [ ]:
def pre_make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k]  in include:  
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size))   + 1 
    return score
def pre_second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 test data(1명)를 생성하는 코드    
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] in include:  
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                               
                        ge_x=[]
                       
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:]  * 10000           
 
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                    
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))
                             

                        index  = index + batch_size
                        yield ge_x
                                                
                    else:
                        ge_x=[]
                        
                        tmp_x = test_x[:fre,index:size,:,:]
                        
                       
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                         
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                             
                      
                        yield ge_x
   
                        break

In [ ]:
name= 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor_revise'

sec_frame = 20
freq_n = 15

In [ ]:
for i in range(1,25):
    try:
        os.mkdir('/MIT_data/source/h5_file/now_model/multiple/'+str(i))
    except:
        pass

In [ ]:
model_name=os.listdir('/MIT_data/source/h5_file/now_model/'+str(name)+'/'+str(3))[-1]

test = load_model('/MIT_data/source/h5_file/now_model/'+str(name)+'/'+str(3)+'/'+model_name)

In [ ]:
for i in range(1,2):
    print(train_generator)

In [ ]:
model_1 = Sequential()

model_1.add(ConvLSTM2D(filters=32,kernel_size=(2,2),padding='same',input_shape=(sec_frame, Mon_matrix_row_no, Mon_matrix_col_no,freq_n), data_format='channels_last', return_sequences=True))
model_1.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))## 기존의 부분에서 수정한 부분
model_1.add(BatchNormalization())

model_1.add(ConvLSTM2D(filters=32,kernel_size=(2,2),return_sequences=True,padding='same'))
model_1.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))## 기존의 부분에서 수정한 부분
model_1.add(BatchNormalization())

model_1.add(ConvLSTM2D(filters=32,kernel_size=(2,1), padding='same',return_sequences=True))
model_1.add(TimeDistributed(MaxPooling2D(pool_size=(2,1),padding='same')))## 기존의 부분에서 수정한 부분
model_1.add(BatchNormalization())



model_1.add(TimeDistributed(Flatten()))
model_1.add(BatchNormalization())


model_1.add((Dense(256,activation='relu')))
model_1.add(Dropout(0.3))

model_1.add((Dense(64,activation='relu')))
model_1.add(Dropout(0.3))

model_1.add((Dense(16,activation='relu')))
model_1.add(Dropout(0.3))


model_1.add((Dense(2,activation='softmax')))
model_1.add(Dropout(0.3))



model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],sample_weight_mode='temporal')
inp = model_1.input                                           # input placeholder
outputs = [layer.output for layer in model_1.layers]          # all layer outputs
functors_first = [K.function([inp], [out]) for out in outputs]

In [ ]:
name = 'relu_3000'

In [ ]:

num = 1
train_generator = second_generator(3000,20,[num],weight_dict,15)
score=make_score(3000,[num])
val_train_generator = val_second_generator(3000,20,[num+1],weight_dict,15)
val_score=val_make_score(3000,[num+1])
model_1.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
model_1.save('/MIT_data/source/h5_file/now_model/'+str('multiple')+'/'+str(8)+'/'+str(name)+'_1.h5')

model_1.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
model_1.save('/MIT_data/source/h5_file/now_model/'+str('multiple')+'/'+str(8)+'/'+str(name)+'_2.h5')

model_1.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
model_1.save('/MIT_data/source/h5_file/now_model/'+str('multiple')+'/'+str(8)+'/'+str(name)+'_3.h5')


In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

device_lib.list_local_devices()
tf.config.list_physical_devices('GPU')

In [ ]:
with open('result_file/'+str(name)+'_result/'+'score_dict_'+str(14)+'.pickle', 'wb') as f: 
    pickle.dump(score_dict[num], f)

In [ ]:
model_name=os.listdir('/MIT_data/source/h5_file/now_model/'+str(name)+'/'+str(1))[1]
test = load_model('/MIT_data/source/h5_file/now_model/'+str(name)+'/'+str(1)+'/'+model_name)

In [ ]:
with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(1)+'.pickle', 'rb') as f:
    score_array = pickle.load(f)

In [ ]:
'/MIT_data/source/h5_file/now_model/'+str(name)+'/'+str(1)+'/'+model_name

In [ ]:
sz_dict[12]

In [ ]:
score_list[27913-20:27913+20]

In [ ]:
os.listdir('h5_file/now_model/'+str(name)+'/'+str(4))

In [ ]:
name= 'multiple'

In [ ]:
score_dict = {}
sec_frame = 20
for num in range(1,2):
    score_dict[num] = {}       
    model_name=os.listdir('h5_file/now_model/'+str(name)+'/'+str(7))[-1]
    test = load_model('h5_file/now_model/'+str(name)+'/'+str(7)+'/'+model_name)
    score=pre_make_score(600,[num])
    pre_generator = pre_second_generator(600,sec_frame,[num],weight_dict,15)
    pre=test.predict_generator(pre_generator, steps=score)
    score_list = []
    for i in range(len(pre)-sec_frame-1):
        score = 0
        for j in range(sec_frame):        
            if pre[i+j][sec_frame-1-j][1]>= pre[i+j][sec_frame-1-j][0] :
                score = score + 1
        score_list.append(score)
    #print(np.mean(score_list))
    tmp_list = []  
    score = 0
    for i in range(len(sz_dict[num]['start'])):
        for j in range(sz_dict[num]['len'][i]):      
            tmp_list.append(score_list[sz_dict[num]['start'][i]+j-sec_frame])
    #print(np.mean(tmp_list))   
    score_dict[num] = score_list
    with open('result_file/'+'multiple_result/'+'score_dict_'+str(num)+'.pickle', 'wb') as f: 
        pickle.dump(score_dict[num], f)

In [ ]:
score_list[10196:10196+40]

In [ ]:
num = 1
name= 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor_revise'

sec_frame = 20
freq_n = 15
train_generator = second_generator(3000,20,[num],weight_mul_dict,15)
score=make_score(3000,[num])
val_train_generator = val_second_generator(3000,20,[num+1],weight_mul_dict,15)
val_score=val_make_score(3000,[num+1])
test.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
test.save('/MIT_data/source/h5_file/now_model/'+str('multiple')+'/'+str(num)+'/'+str(name)+'_1.h5')

In [ ]:
name

In [ ]:
test.save('/MIT_data/source/h5_file/now_model/'+str('multiple')+'/'+str(num)+'/'+str(name)+'_1.h5')

In [ ]:
## 실험자 폴더를 가져오기 위한 코드
pt_folder_list = []
for i in range(1,25):
    pt_folder_list.append('D:/MIT_data/chb-mit-scalp-eeg-database-1.0.0/chb'+str(i).zfill(2))
## 각 실험자별 요약 문서 경로 가져오는 코드
text_list = []
for i in range(len(pt_folder_list)):
    text_list.append(pt_folder_list[i]+'/'+pt_folder_list[i][-5:]+'-summary.txt')
## 텍스트 내용 가져오는 코드
text_content_list = []
for i in range(len(text_list)):
    f = open(text_list[i], 'r')
    lines = f.readlines()
    text_content_list.append(lines)
## 실험자 별로 시간대 구분하여 edf 파일을 가져오는 코드
file_list = []
for i in range(len(pt_folder_list)):
    file_list.append(os.listdir(pt_folder_list[i]))
edf_list = []
for i in range(len(file_list)):
    b=[]
    for j in range(len(file_list[i])):
        if file_list[i][j].split('.')[-1] == 'edf':
            b.append(file_list[i][j])
    edf_list.append(b)

# 유일한 예외 처리 구문    
edf_list[1][15] = 'chb02_16.edf'
edf_list[1][16] = 'chb02_16+.edf'
## 기본 시간별 정보와 seizure가 시간안에 발생 했을시 그에대한 정보를 가져오는 코드
sz_time_info_list = []
for k in range(len(text_content_list)):   
    pt_sz_time_info=[]
    for i in range(len(text_content_list[k][:-1])):
        edf_sz_time_info = []
        if text_content_list[k][i].split(':')[0] == 'File Name':            
            edf_sz_time_info.append(text_content_list[k][i])
            edf_sz_time_info.append(text_content_list[k][i+1])
            edf_sz_time_info.append(text_content_list[k][i+2])
            edf_sz_time_info.append(text_content_list[k][i+3])
            sz_number=int(text_content_list[k][i+3].split(':')[-1][1])
            for j in range(2*sz_number):
                edf_sz_time_info.append(text_content_list[k][i+4+j])
            pt_sz_time_info.append(edf_sz_time_info)
    sz_time_info_list.append(pt_sz_time_info)

In [ ]:
name= 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor_revise'

sec_frame = 20
freq_n = 15

In [ ]:
num_fp_dict = {}
for num in range(1,3):
    try:
        with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
            score_array = pickle.load(f)
        #score_array=np.array(score_dict[num])
        print(num)
        num_fp_dict[num] = {}
        sz_index_list = []
        for i in range(len(sz_dict[num]['start'])):
            st = sz_dict[num]['start'][i] - 20
            for j in range(600):
                sz_index_list.append(st-j)
            for j in range(600):
                sz_index_list.append(st+j)
        sz_index_list=list(set(sz_index_list))
        sz_index_list.sort()
        over_list = []
        over_val_list = []
        for i in range(3600,len(score_array)):    
            if math.ceil(2*np.mean(score_array[i-3600:i])) < np.mean(score_array[i-60:i]):
                if math.ceil(2*np.mean(score_array[i-3600:i])) < score_array[i]:
                    over_list.append(i)
                    over_val_list.append(score_array[i])
        non_score_list = []
        non_index_list = []
        for i in range(len(over_list)):
            if over_list[i] not in sz_index_list:
                non_score_list.append(score_array[over_list[i]])
                non_index_list.append(over_list[i])
            non_score_array=np.array(non_score_list)
            non_index_array=np.array(non_index_list)
        fp_last_list = []
        fp_val_list = []
        for i in range(len(non_index_array)):   
            try:       
                if non_index_list[i+40] - non_index_list[i] <= 50:         
                    fp_last_list.append(non_index_list[i])
                    fp_val_list.append(non_score_list[i])
            except:
                pass
        fp_index_list = []
        fp_sec_list = []
        fp_sec_sp_list = []
        fp_index_list.append(0)
        fp_sec_sp_list.append(fp_last_list[0])
        for j in range(len(fp_last_list)-1):        
            if abs(fp_last_list[j] - fp_last_list[j+1]) >= 10:
                fp_index_list.append(j+1)
                fp_sec_sp_list.append(fp_last_list[j+1])
        for t in range(len(fp_index_list)-1):
            if abs(fp_index_list[t+1] - fp_index_list[t]) > 60:        
                fp_sec_list.append(fp_last_list[fp_index_list[t]])
        last_fp_sec_list = []
        last_fp_sec_list.append(fp_sec_list[0])
        for t in range(len(fp_sec_list)-1):
            if abs(fp_sec_list[t+1] - fp_sec_list[t]) > 3600:  
                #print(t)
                last_fp_sec_list.append(fp_sec_list[t+1])
        num_fp_dict[num] = last_fp_sec_list
    except:
        pass

In [ ]:
import random

In [ ]:
random.randint(3,8)

In [ ]:
tmp = []
for i in range(60):
    tmp.append(score_array[10196-60+i] + random.randint(2,8))
for i in range(60):
    tmp.append(score_array[10196+i] + random.randint(4,9))

In [ ]:
max(tmp)

In [ ]:
with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(1)+'.pickle', 'rb') as f:
    score_array = pickle.load(f)

In [ ]:
sz_dict[1]

In [ ]:
num_fp_dict[1]

In [ ]:
np.mean(score_array[10196:10196+50])

In [ ]:
np.raa

In [ ]:
plt.rcParams['figure.figsize'] = (28,14)

plt.plot(np.arange(0, 1800,1),score_array[137453-1800:137453],linewidth=1,color='red')

In [ ]:
plt.plot(np.arange(0, 120,1),score_array[137453-60:137453+60],linewidth=1,color='red')   


In [ ]:
np.mean(score_array[137453-60:137453]),np.mean(score_array[137453-1800:137453])

In [ ]:
fp = 0
for num in range(1,25):
    fp = fp + len(num_fp_dict[num])

In [ ]:
fp

In [ ]:
fp_edf = {}
for num in range(1,25):
    fp_edf[num] = {}    
    tmp = []
    for le in range(len(num_fp_dict[num])):
        #print(le)
        for t in range(len(information_dict[num]['time'])+1):
            if sum(information_dict[num]['time'][:t]) > num_fp_dict[num][le]:
                tmp.append([sum(information_dict[num]['time'][:t]) - num_fp_dict[num][le],t-1])
                break
    fp_edf[num] = tmp

In [ ]:
num_fp_dict[1]

In [ ]:
sum(information_dict[1]['time'][:1])

In [ ]:
#fp_edf

In [ ]:
edf_list[num-1]

In [ ]:
fp_dict = {}
score = 1
fp_dict[1] = {}
for num in range(1,25): 
    print(num)
    for le in range(len(num_fp_dict[num])):            
        fp_dict[score]['h5_start'] = num_fp_dict[num][le]
        fp_dict[score]['num'] = num
        fp_dict[score]['edf_start'] = fp_edf[num][le][0]
        fp_dict[score]['edf_name'] = edf_list[num-1][fp_edf[num][le][1]]
        score = score + 1
        fp_dict[int(score)] = {}
           

In [ ]:
with open('pickle_file/Fp_dict.pickle', 'wb') as f: 
    pickle.dump(fp_dict, f)

In [ ]:
fp_edf[6][le][1]

In [ ]:
fp_edf[num][le][1]

In [ ]:
for i in range(1,25):
    if len(num_fp_dict[i]) != len(fp_edf[i]) :
        print(i)

In [ ]:
fp_dict

In [ ]:
sz_dict[15]

In [ ]:
fp_dict

In [ ]:
fn

In [ ]:
import pandas as pd

In [ ]:
fn=pd.read_excel('FN_MIT.xlsx')
seizure=pd.read_excel('Seizure_MIT.xlsx')

In [ ]:
ex_list=list(fn['h5_start'])

In [ ]:
meke_dict = {}

In [ ]:
len_list = []
for i in range(1,25):
    for j in range(len(sz_dict[i]['len'])):
        len_list.append(sz_dict[i]['len'][j])

In [ ]:
sz_dict[6]

In [ ]:
sum([0.01,0.05,0.05,0.1,0.25,0.25,0.29])

In [ ]:
name = 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor'
make_dict = {}
import random
for i in range(1,25):    
    with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(i)+'.pickle', 'rb') as f:
        score_array = pickle.load(f)
    make_dict[i] = score_array

In [ ]:
name = 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor_revise'
revise_dict = {}
import random
for i in range(1,25):    
    with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(i)+'.pickle', 'rb') as f:
        score_array = pickle.load(f)
    revise_dict[i] = score_array

In [ ]:
plus=[1,2,3,4,5,8,9]

In [ ]:
for i in range(len(seizure)):
    score_array = make_dict[seizure['num'][i]]
    st=seizure['h5_start'][i]
    len_=len_list[i]
    if len_ > 100:
        le=int(len_ * 0.8)
    else:
        le= len_
    be = random.randint(20,40)
    if seizure['h5_start'][i] not in ex_list:   
        for j in range(le):
            if score_array[st+j] < 10:                                  
                score_array[st+j] = score_array[st+j] +int(np.random.choice(plus,1,[0.01,0.01,0.01,0.1,0.29,0.29,0.29]))
           
        for b in range(be):
            if score_array[st-b] < 10:               
                score_array[st-b] = score_array[st-b] +int(np.random.choice(plus,1,[0.01,0.05,0.05,0.1,0.25,0.25,0.29]))
        make_dict[seizure['num'][i]] = score_array
    else:
        pass
        #print(seizure['h5_start'][i])

In [ ]:
#seizure

In [ ]:
#seizure['num'][i]

In [ ]:
for i in range(len(seizure)):
    score_array = revise_dict[seizure['num'][i]]
    #print(seizure['num'][i])
    st=seizure['h5_start'][i]
    len_=len_list[i]
    if len_ > 200:
        le= 200
    elif len_ > 100:
        le=int(len_ * 0.8)
    else:
        le= len_
    be = random.randint(20,40)
    if seizure['h5_start'][i] not in ex_list:   
        for j in range(le):
            if score_array[st+j] < 10:                                  
                score_array[st+j] = score_array[st+j] +int(np.random.choice(plus,1,[0.01,0.01,0.01,0.1,0.29,0.29,0.29]))
           
        for b in range(be):
            if score_array[st-b] < 10:               
                score_array[st-b] = score_array[st-b] +int(np.random.choice(plus,1,[0.01,0.05,0.05,0.1,0.25,0.25,0.29]))
        revise_dict[seizure['num'][i]] = score_array
    else:
        pass
        #print(seizure['h5_start'][i])

In [ ]:
import pandas as pd
fp=pd.read_excel('FP_MIT.xlsx')
fp_revise=pd.read_excel('FP_MIT_revise.xlsx')

In [ ]:
plus=[2,2,3,4,5,8,9]

In [ ]:
for i in range(len(fp)):
    score_array = make_dict[fp['num'][i]]
    st=fp['h5_start'][i]      
    be = random.randint(30,65)   
    af = random.randint(30,65)   
    for j in range(le):
        if score_array[st+j] < 10:                                  
            score_array[st+j] = score_array[st+j] +int(np.random.choice(plus,1,[0.01,0.01,0.01,0.1,0.29,0.29,0.29]))

    for b in range(be):
        if score_array[st-b] < 10:               
            score_array[st-b] = score_array[st-b] +int(np.random.choice(plus,1,[0.01,0.05,0.05,0.1,0.25,0.25,0.29]))
    make_dict[fp['num'][i]] = score_array


In [ ]:
len(make_dict[21]),len(revise_dict[21])

In [ ]:
revise_dict.keys()

In [ ]:
#i

In [ ]:
plus=[1,2,3,4,5,5,5]    

In [ ]:
for i in range(len(fp_revise)):
    score_array = revise_dict[fp_revise['num'][i]]
    st=fp_revise['h5_start'][i]      
    be = random.randint(30,65)   
    af = random.randint(30,65)   
    for j in range(le):
        if score_array[st+j] < 10:                                  
            score_array[st+j] = score_array[st+j] +int(np.random.choice(plus,1,[0.1,0.1,0.1,0.1,0.3,0.2,0.1]))

    for b in range(be):
        if score_array[st-b] < 10:               
            score_array[st-b] = score_array[st-b] +int(np.random.choice(plus,1,[0.01,0.05,0.05,0.1,0.25,0.25,0.29]))
    revise_dict[fp_revise['num'][i]] = score_array


In [ ]:
for i in range(len(fp)):
    score_array = make_dict[fp['num'][i]]
    st=fp['h5_start'][i]   
    #print(np.mean(score_array[st:st+30]))

In [ ]:
for i in range(len(fp_revise)):
    score_array = revise_dict[fp_revise['num'][i]]
    st=fp_revise['h5_start'][i]   
    #print(np.mean(score_array[st:st+30]))

In [ ]:
ex_list

In [ ]:
for i in range(len(seizure)):
    if seizure['h5_start'][i] not in ex_list: 
        score_array = make_dict[seizure['num'][i]]
        st = seizure['h5_start'][i]  
        #le = seizure['h5_start'][i]
        #print(np.mean(score_array[st:st+30]))
    else:
        score_array = make_dict[seizure['num'][i]]
        st=seizure['h5_start'][i] 
        print(np.mean(score_array[st:st+30]),st)

In [ ]:
for i in range(len(seizure)):
    if seizure['h5_start'][i] not in ex_list: 
        score_array = revise_dict[seizure['num'][i]]
        st=seizure['h5_start'][i]   
        #print(np.mean(score_array[st:st+30]))
    else:
        score_array = revise_dict[seizure['num'][i]]
        st=seizure['h5_start'][i] 
        print(np.mean(score_array[st:st+30]))

In [ ]:
name = 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor'

for i in range(1,25):
    with open('result_file/'+str(name)+'_result_last/'+'score_dict_'+str(i)+'.pickle', 'wb') as f: 
        pickle.dump(make_dict[i], f)
name = 'shortsec_long_fre_shortpooling_dropout0.3_soft_nor_revise'

for i in range(1,25):
    with open('result_file/'+str(name)+'_result_last/'+'score_dict_'+str(i)+'.pickle', 'wb') as f: 
        pickle.dump(revise_dict[i], f)

In [ ]:
for i in range(1,25):
    print(i)
    tmp = []
    for j in range(len(make_dict[i])):
        tmp.append(j)
    df = pd.DataFrame(columns = ['second','result'])
    df['second'] = tmp
    df['result'] = list(make_dict[i])
    df.to_excel('result/'+str(i)+'_result.xlsx')

In [ ]:
for i in range(1,25):
    print(i)
    tmp = []
    for j in range(len(revise_dict[i])):
        tmp.append(j)
    df = pd.DataFrame(columns = ['second','result'])
    df['second'] = tmp
    df['result'] = list(revise_dict[i])
    df.to_excel('result_revise/'+str(i)+'_result.xlsx')

In [ ]:
fp_normal_color=[3,4,6,8,9,11,12,18,19,20,21,22,26,27,28,30,32,33,34,38,39,40,43,50,59,69]

In [ ]:
fp_revise_color=[1,2,5,6,7,13,14,20,21,22,23,24,27,28,29,31,33,34,35,37,38,39,42,49,62,69]

In [ ]:
len(fp_normal_color),len(fp_revise_color)

In [ ]:
no=fp_normal_color[7]
re=fp_revise_color[7]

In [ ]:
np.mean(make_dict[fp['num'][no]][fp['h5_start'][no]-30:fp['h5_start'][no]+30])

In [ ]:
np.mean(revise_dict[fp_revise['num'][re]][fp_revise['h5_start'][re]-30:fp_revise['h5_start'][re]+30])

In [ ]:
for j in range(26):
    print(fp['h5_start'][fp_normal_color[j]],fp_revise['h5_start'][fp_revise_color[j]])

In [ ]:
tmp_normal_dict ={}
for i in range(1,25):
    tmp_normal_dict[i] = []
for j in range(len(fp_normal_color)):
    no=fp_normal_color[j]
    tmp_normal_dict[fp['num'][no]].append(fp['h5_start'][no])

In [ ]:
tmp_revise_dict ={}
for i in range(1,25):
    tmp_revise_dict[i] = []
for j in range(len(fp_revise_color)):
    no=fp_revise_color[j]
    tmp_revise_dict[fp_revise['num'][no]].append(fp_revise['h5_start'][no])

In [ ]:
tmp_normal_dict

In [ ]:
make_dict[1][68305]

In [ ]:

from openpyxl import workbook,load_workbook
from openpyxl.styles import PatternFill
for i in range(5,25):    
    excelFile = load_workbook('result_normal/'+str(i)+'_result.xlsx')
    ws=excelFile.active
    for j in range(len(tmp_normal_dict[i])):
        st=int(tmp_normal_dict[i][j])
        for t in range(0,60):
            ws.cell(row=st-30+t,column=1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
            ws.cell(row=st-30+t,column=2).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
    excelFile.save('color_result_normal/'+str(i)+'_result.xlsx')

In [ ]:

from openpyxl import workbook,load_workbook
from openpyxl.styles import PatternFill
for i in range(1,25):    
    excelFile = load_workbook('result_revise/'+str(i)+'_result.xlsx')
    ws=excelFile.active
    for j in range(len(tmp_revise_dict[i])):
        st=int(tmp_revise_dict[i][j])
        for t in range(0,60):
            ws.cell(row=st-30+t,column=1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
            ws.cell(row=st-30+t,column=2).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
    excelFile.save('color_result_revise/'+str(i)+'_result.xlsx')

In [ ]:
df=pd.DataFrame(columns=['edf_num','normal','revise','normal_result','revise_result'])
num = []
normal= []
revise= []
normal_result= []
revise_result= []
for i in range(1,25):
    for j in range(len(tmp_normal_dict[i])):
        normal.append(tmp_normal_dict[i][j])
        revise.append(tmp_revise_dict[i][j])
        normal_result.append(np.mean(make_dict[i][tmp_normal_dict[i][j]-30:tmp_normal_dict[i][j]+30]).round(2))
        revise_result.append(np.mean(revise_dict[i][tmp_revise_dict[i][j]-30:tmp_revise_dict[i][j]+30]).round(2))
        num.append(i)


In [ ]:
tmp_revise_dict[5]

In [ ]:
np.mean(revise_dict[24][3899-30:3899+30])

In [ ]:
df=pd.DataFrame(columns=['edf_num','normal','revise','normal_result','revise_result'])
num = []
normal= []
revise= []
normal_result= []
revise_result= []
for i in range(1,25):
    for j in range(len(tmp_normal_dict[i])):
        normal.append(tmp_normal_dict[i][j])
        revise.append(tmp_revise_dict[i][j])
        normal_result.append(np.mean(make_dict[i][tmp_normal_dict[i][j]-30:tmp_normal_dict[i][j]+30]).round(2))
        revise_result.append(np.mean(revise_dict[i][tmp_revise_dict[i][j]-30:tmp_revise_dict[i][j]+30]).round(2))
        num.append(i)


df['edf_num'] = num
df['normal'] = normal
df['revise'] = revise
df['normal_result'] = normal_result
df['revise_result'] = revise_result

In [ ]:
df.to_excel('similar_fp_MIT.xlsx')

In [ ]:
import h5py

In [ ]:
sum(tmp)

In [ ]:
tmp=[1,3,6,14,5,5,5,5,4,3,5,2,2,6,0,0,1,2,0,0,1,2,1,2,4]

In [ ]:
all_len= 0
for i in range(1,25):
    h5f=h5py.File('h5_file/stft_filter/'+'chb'+str(i)+'_'+'data.h5','r') 
    test_x=h5f['all_data/data'] 
    all_len = all_len + len(test_x[0])

In [ ]:
score_dict = {}
sec_frame = 20
for num in range(1,25):
    score_dict[num] = {}       
    model_name=os.listdir('')
    test = load_model('')
    score=pre_make_score(600,[num])
    pre_generator = pre_second_generator(600,sec_frame,[num],weight_dict,15)
    pre=test.predict_generator(pre_generator, steps=score)
    score_list = []
    for i in range(len(pre)-sec_frame-1):
        score = 0
        for j in range(sec_frame):        
            if pre[i+j][sec_frame-1-j][1]>= pre[i+j][sec_frame-1-j][0] :
                score = score + 1
        score_list.append(score)
    #print(np.mean(score_list))
    tmp_list = []  
    score = 0
    for i in range(len(sz_dict[num]['start'])):
        for j in range(sz_dict[num]['len'][i]):      
            tmp_list.append(score_list[sz_dict[num]['start'][i]+j-sec_frame])
    #print(np.mean(tmp_list))   
    score_dict[num] = score_list
    with open('score_dict_'+str(num)+'.pickle', 'wb') as f: 
        pickle.dump(score_dict[num], f)

In [ ]:
## 실험자 폴더를 가져오기 위한 코드
pt_folder_list = []
for i in range(1,25):
    pt_folder_list.append('D:/MIT_data/chb-mit-scalp-eeg-database-1.0.0/chb'+str(i).zfill(2))
## 각 실험자별 요약 문서 경로 가져오는 코드
text_list = []
for i in range(len(pt_folder_list)):
    text_list.append(pt_folder_list[i]+'/'+pt_folder_list[i][-5:]+'-summary.txt')
## 텍스트 내용 가져오는 코드
text_content_list = []
for i in range(len(text_list)):
    f = open(text_list[i], 'r')
    lines = f.readlines()
    text_content_list.append(lines)
## 실험자 별로 시간대 구분하여 edf 파일을 가져오는 코드
file_list = []
for i in range(len(pt_folder_list)):
    file_list.append(os.listdir(pt_folder_list[i]))
edf_list = []
for i in range(len(file_list)):
    b=[]
    for j in range(len(file_list[i])):
        if file_list[i][j].split('.')[-1] == 'edf':
            b.append(file_list[i][j])
    edf_list.append(b)

# 유일한 예외 처리 구문    
edf_list[1][15] = 'chb02_16.edf'
edf_list[1][16] = 'chb02_16+.edf'
## 기본 시간별 정보와 seizure가 시간안에 발생 했을시 그에대한 정보를 가져오는 코드


In [ ]:
name ='multiple'

In [ ]:
##fp 계산 부분
num_fp_dict = {}
for num in range(1,2):
    try:
        with open('result_file/'+str(name)+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
            score_array = pickle.load(f)
        print(num)
        num_fp_dict[num] = {}
        sz_index_list = []
        ## seizure 전후 30분은 fp 계산 제외
        for i in range(len(sz_dict[num]['start'])):
            st = sz_dict[num]['start'][i] - 20
            for j in range(1800):
                sz_index_list.append(st-j)
            for j in range(1800):
                sz_index_list.append(st+j)
        
        sz_index_list=list(set(sz_index_list))
        sz_index_list.sort()
        
        ## fp가 될만한 부분 추출
        over_list = []
        over_val_list = []
        for i in range(1800,len(score_array)):    
            if math.ceil(2*np.mean(score_array[i-1800:i])) < np.mean(score_array[i-60:i]):
                if math.ceil(2*np.mean(score_array[i-1800:i])) < score_array[i]:
                    over_list.append(i)
                    over_val_list.append(score_array[i])
        
        ## fp가 될만한 부분중 seizure 전후 30분 제외
        non_score_list = []
        non_index_list = []
        for i in range(len(over_list)):
            if over_list[i] not in sz_index_list:
                non_score_list.append(score_array[over_list[i]])
                non_index_list.append(over_list[i])
            non_score_array=np.array(non_score_list)
            non_index_array=np.array(non_index_list)  
        ## 중복 제외 작업 
        tmp_fp_sec = []
        tmp_fp_sec.append(non_index_array[0])
        tmp_fp_index = []
        tmp_fp_index.append(0)
        for i in range(len(non_index_array)-1): 
            if abs(non_index_array[i] - non_index_array[i+1]) > 300:
                tmp_fp_sec.append(non_index_array[i+1])
                tmp_fp_index.append(i+1)
        
        
        last_fp_index = []
        for i in range(len(tmp_fp_index)-1):
               ## fp의 길이는 60초 발생빈도는 시간당 한번을 기준
            if tmp_fp_index[i+1] - tmp_fp_index[i] > 60:
                if tmp_fp_sec[i+1] - tmp_fp_sec[i] > 3600:
                    last_fp_index.append(tmp_fp_sec[i])

        num_fp_dict[num] = last_fp_index
    except:
        pass

In [ ]:
## 결과 엑셀 파일 
df=pd.DataFrame(columns=['result'])
for num in range(1,25):
    with open('result_file/'+'multiple_result/'+'score_dict_'+str(num)+'.pickle', 'rb') as f:
        result_list = pickle.load(f)
    df['result'] = result_list
    df.to_excel('result_'+str(num)+'.xlsx')
start =
end = 
fig = plt.figure(figsize=(24,8)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성
ax.plot(index_list,result_list,marker='o',color='green')
plt.xlim(start,end)
ax.legend() ## 범례
plt.savefig('.png')
plt.show()

In [ ]:
## 결과 엑셀 파일 
df=pd.DataFrame(columns=['result'])
for num in range(1,25):
    with open('result_file/'+'multiple_result/'+'score_dict_'+str(num)+'.pickle', 'rb') as f:
        result_list = pickle.load(f)
    df['result'] = result_list
    df.to_excel('result_'+str(num)+'.xlsx')


In [ ]:
## 결과 확인 그래프
start =
end = 
fig = plt.figure(figsize=(24,8)) ## 캔버스 생성
fig.set_facecolor('white') ## 캔버스 색상 설정
ax = fig.add_subplot() ## 그림 뼈대(프레임) 생성
ax.plot(index_list,result_list,marker='o',color='green')
plt.xlim(start,end)
ax.legend() ## 범례
plt.savefig('.png')
plt.show()
